In [1]:
print('Downloading data')
print('...')
print()
!wget http://hck.re/kkBIfM
print('Data successfully downloaded')
print('##########################################################')

print('Loading Libraries..')
import zipfile
import os,shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
import warnings
warnings.filterwarnings('always')





print('EXTRACTING FILES')
with zipfile.ZipFile('/content/kkBIfM') as f:
  f.extractall()
  
with zipfile.ZipFile('/content/DataSet/Train Images.zip') as f:
  f.extractall()
print('# of train images:', len(os.listdir('/content/Train Images/Large'))+\
      len(os.listdir('/content/Train Images/Small'))
     )
  
with zipfile.ZipFile('/content/DataSet/Test Images.zip') as f:
  f.extractall()
  
print('# of test images:',len(os.listdir('/content/Test Images')))
print()

print('Creating Validation dir.')
print()
train_dir = '/content/Train Images/'
test_dir = '/content/Test Images/'
!mkdir '/content/Validation Images'
val_dir = '/content/Validation Images/'
!mkdir '/content/Validation Images/Large/'
!mkdir '/content/Validation Images/Small/'
index = np.random.choice(range(5999),size=[1000],replace=False)
val_small = np.array(os.listdir(train_dir+'Small'))[index]
#val_small
val_large =  np.array(os.listdir(train_dir+'Large'))[index]


for i in val_small:
  shutil.move(train_dir+'Small/'+i,val_dir+'Small/'+i)
  
for i in val_large:
  shutil.move(train_dir+'Large/'+i,val_dir+'Large/'+i)
  


print('Loading train generator')
print()

#train_gen = ImageDataGenerator(rescale=1./255.)
#train_gen = ImageDataGenerator(rescale=1./255)
train_gen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

train_generator = train_gen.flow_from_directory(
        train_dir, 
        target_size=(300,400 ),
        shuffle = True,
        batch_size=50,
        class_mode='binary')

print('Loading validation generator')
print()

val_gen = ImageDataGenerator(rescale=1./255.)
val_generator = train_gen.flow_from_directory(
        val_dir, 
        target_size=(300,400),
        shuffle = True,
        batch_size=50,
        class_mode='binary')

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(300,400, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
   tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(1, activation='sigmoid')
])



print('Hello world')
print()

model.compile(optimizer = RMSprop(lr=0.001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])
print('Model compiled')
print()

history = model.fit_generator(
      train_generator,
      validation_data= val_generator,   
      steps_per_epoch=20,  
      epochs=10,
      verbose=2)
print('Model training completed')
print('##########################################################')
print('Validation; objective: F1 score')
print()
filenames = val_generator.filenames
nb_samples = len(filenames)
print('Prediction')
#predict = model.predict_generator( val_generator,np.ceil(nb_samples/32))
predict = model.predict_generator(val_generator)
predicted_classes = [1 if i>0.5 else 0 for i in predict]
true_y = val_generator.filenames
true_y = [i.split('/')[0] for i in true_y]
true_y = [0 if i=='Large' else 1 for i in true_y]
from sklearn.metrics import f1_score,accuracy_score,precision_score,roc_auc_score
print('f1 score = ',100*f1_score(true_y,predicted_classes))


    


    




...

--2019-10-30 22:33:10--  http://hck.re/kkBIfM
Resolving hck.re (hck.re)... 54.169.227.143, 18.136.192.20, 54.169.47.64
Connecting to hck.re (hck.re)|54.169.227.143|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Cookie coming from hck.re attempted to set domain to hackerearth.com
Cookie coming from hck.re attempted to set domain to hackerearth.com
Cookie coming from hck.re attempted to set domain to hackerearth.com
Location: https://he-public-data.s3-ap-southeast-1.amazonaws.com/DataSet.zip [following]
--2019-10-30 22:33:16--  https://he-public-data.s3-ap-southeast-1.amazonaws.com/DataSet.zip
Resolving he-public-data.s3-ap-southeast-1.amazonaws.com (he-public-data.s3-ap-southeast-1.amazonaws.com)... 52.219.48.51
Connecting to he-public-data.s3-ap-southeast-1.amazonaws.com (he-public-data.s3-ap-southeast-1.amazonaws.com)|52.219.48.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733209860 (699M) [application/zip]
Saving to

EXTRACTING FILES
# of train images: 11998
# of test images: 7534

Creating Validation dir.

Loading train generator

Found 9998 images belonging to 2 classes.
Loading validation generator

Found 2000 images belonging to 2 classes.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Hello world

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model compiled

Epoch 1/10
Epoch 1/10
40/20 - 78s - loss: 0.5560 - acc: 0.7630
20/20 - 113s - loss: 0.8656 - acc: 0.6320 - val_loss: 0.5504 - val_acc: 0.7630
Epoch 2/10
Epoch 1/10
40/20 - 77s - loss: 0.4371 - acc: 0.8225
20/20 - 96s - loss: 0.5233 - acc: 0.7670 - val_loss: 0.4323 - val_acc: 0.8225
Epoch 3/10
Epoch 1/10
40/20 - 77s - loss: 0.2590 - acc: 0.9015
20/20 - 96s - loss: 0.4702 - acc: 0.8317 - val_loss: 0.2453 - val_acc: 0.9015
Epoch 4/10
Epoch 1/10
40/20 - 78s - loss: 0.2716 - acc: 0.9330
20/20 - 95s - loss: 0.5834 - acc: 0.8970 - val_loss: 0.2712 - val_acc: 0.9330


/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(

f1 score =  49.235474006116206


/usr/local/lib/python3.6/dist-packages/_pytest/mark/structures.py:426: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [9]:
from tensorflow.keras.optimizers import Adam, SGD
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,400, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3),padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
   tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3),padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(1, activation='sigmoid')
])



print('Hello world')
print()
sgd = SGD(nesterov=True)
rms = RMSprop(lr=0.001)
adam = Adam()
model.compile(optimizer = adam, 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])
print('Model compiled')
print()

history = model.fit_generator(
      train_generator,
      validation_data= val_generator,   
      steps_per_epoch=20,  
      epochs=20,
      verbose=2)
print('Model training completed')
print('##########################################################')
print('Validation; objective: F1 score')
print()
filenames = val_generator.filenames
nb_samples = len(filenames)
print('Prediction')
#predict = model.predict_generator( val_generator,np.ceil(nb_samples/32))
predict = model.predict_generator(val_generator)
predicted_classes = [1 if i>0.5 else 0 for i in predict]
true_y = val_generator.filenames
true_y = [i.split('/')[0] for i in true_y]
true_y = [0 if i=='Large' else 1 for i in true_y]
from sklearn.metrics import f1_score,accuracy_score,precision_score,roc_auc_score
print('f1 score = ',100*f1_score(true_y,predicted_classes))


    


    

Hello world

Model compiled

Epoch 1/20
Epoch 1/20
40/20 - 78s - loss: 0.5766 - acc: 0.7525
20/20 - 111s - loss: 0.6387 - acc: 0.6080 - val_loss: 0.5370 - val_acc: 0.7525
Epoch 2/20
Epoch 1/20
40/20 - 77s - loss: 0.2500 - acc: 0.9255
20/20 - 94s - loss: 0.3567 - acc: 0.8570 - val_loss: 0.2252 - val_acc: 0.9255
Epoch 3/20
Epoch 1/20
40/20 - 78s - loss: 0.1589 - acc: 0.9105
20/20 - 95s - loss: 0.1751 - acc: 0.9300 - val_loss: 0.1956 - val_acc: 0.9105
Epoch 4/20
Epoch 1/20
40/20 - 77s - loss: 0.0933 - acc: 0.9640
20/20 - 94s - loss: 0.1208 - acc: 0.9500 - val_loss: 0.1141 - val_acc: 0.9640
Epoch 5/20
Epoch 1/20
40/20 - 78s - loss: 0.1077 - acc: 0.9735
20/20 - 95s - loss: 0.1012 - acc: 0.9619 - val_loss: 0.0926 - val_acc: 0.9735
Epoch 6/20
Epoch 1/20
40/20 - 77s - loss: 0.0797 - acc: 0.9845
20/20 - 94s - loss: 0.0993 - acc: 0.9650 - val_loss: 0.0610 - val_acc: 0.9845
Epoch 7/20
Epoch 1/20
40/20 - 77s - loss: 0.1733 - acc: 0.9625
20/20 - 94s - loss: 0.0891 - acc: 0.9770 - val_loss: 0.1098 -

In [0]:

print('Moving test files to "test" dir.')

print()
test_dir = '/content/Test Images/'

!mkdir '/content/Test Images/test'
print('Test Data generator director has been created; ',test_dir+'test')
print()


for i in os.listdir(test_dir):
  if i!='test':
    shutil.move(test_dir+i,test_dir+'test/'+i)

print('Testing..')
print()
test_dir = '/content/Test Images/'
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        shuffle = False,
        batch_size=32, 
        class_mode=None,)

filenames = test_generator.filenames
nb_samples = len(filenames)
print('Prediction')
predict = model.predict_generator(test_generator,np.ceil(nb_samples/32))
predicted_classes = [1 if i>0.5 else 0 for i in predict]


print('Processing output..')
print()
keys = os.listdir(test_dir+'test')
keys = [i for i in keys if i!='.ipynb_checkpoints']

df = pd.read_csv('/content/DataSet/test.csv')
test_files = df.Image_File.tolist()
_ = []
for i in test_files:
  
  if i in keys:
    _.append(i)
  else:
    break;
    print('ERROR: MISSING FILE !')
    
dic = dict(zip(_,predicted_classes))
df['Class'] = df['Image_File'].apply(lambda x : dic[x])
df.to_csv('submission_file.csv',index=False)
print('Test predictions saved to submission_file.csv')
df


    
